# 策略对比汇总（Classic vs ML）

这个 notebook 用来把：
- 经典策略（MACD/RSI/Bollinger）的回测统计
- 以及 `run_lstm2_pipeline.py` 输出的 ML 回测统计（JSON）

汇总成一张对比表，方便写中期报告。


In [1]:
import json
from pathlib import Path

import numpy as np
import pandas as pd

from fyp_trading.data import fetch_prices
from fyp_trading.backtest import equity_and_stats_from_positions
from fyp_trading.strategies import (
    MacdConfig, macd_crossover_signal,
    RsiConfig, rsi_mean_reversion_signal,
    BollingerConfig, bollinger_mean_reversion_signal,
    DualMaConfig, dual_ma_trend_signal,
    DonchianConfig, donchian_breakout_signal,
    VolTargetMomentumConfig, vol_target_momentum_position,
    BollingerSqueezeConfig, bollinger_squeeze_breakout_signal,
)

ROOT = Path(".").resolve()
REPORTS_DIR = ROOT / "outputs" / "reports"

TICKER = "2800.HK"
PERIOD = "3y"
INTERVAL = "1d"
COST_BP = 2.0
ALLOW_SHORT = False

# 1) classic strategies

df = fetch_prices(TICKER, PERIOD, INTERVAL)
close = df["Close"].copy()
high = df["High"].copy()
low = df["Low"].copy()

next_ret = np.log(close.shift(-1) / close)
simple_ret = (np.exp(next_ret) - 1.0).dropna()

results = []

def add_result(name: str, position: pd.Series):
    _, stats = equity_and_stats_from_positions(simple_ret, position.loc[simple_ret.index], COST_BP)
    results.append({"strategy": name, **stats})

# MACD
add_result("MACD", macd_crossover_signal(close, MacdConfig(allow_short=ALLOW_SHORT)))

# RSI
add_result("RSI", rsi_mean_reversion_signal(close, RsiConfig(allow_short=ALLOW_SHORT)))

# Bollinger mean reversion
add_result("Bollinger(MR)", bollinger_mean_reversion_signal(close, BollingerConfig(allow_short=ALLOW_SHORT)))

# Dual MA trend
add_result("DualMA(10/50)", dual_ma_trend_signal(close, DualMaConfig(fast=10, slow=50, allow_short=ALLOW_SHORT)))

# Donchian breakout
add_result("Donchian(20/10)", donchian_breakout_signal(high, low, DonchianConfig(entry=20, exit=10, allow_short=ALLOW_SHORT)))

# Vol-target momentum (uses continuous position size; allow short = True by default)
add_result(
    "VolTargetMom(60,20)",
    vol_target_momentum_position(
        close,
        VolTargetMomentumConfig(mom_lookback=60, vol_lookback=20, target_vol_annual=0.10, max_leverage=1.0, allow_short=True),
    ),
)

# Bollinger squeeze breakout
add_result(
    "BollSqueeze(20,q0.2)",
    bollinger_squeeze_breakout_signal(close, BollingerSqueezeConfig(period=20, num_std=2.0, squeeze_lookback=120, squeeze_quantile=0.2, allow_short=ALLOW_SHORT)),
)

# 2) ML (read latest json)
# LSTM
ml_jsons = sorted(REPORTS_DIR.glob("lstm2_backtest_stats_*.json"))
if ml_jsons:
    latest = ml_jsons[-1]
    with latest.open("r", encoding="utf-8") as f:
        ml_stats = json.load(f)
    results.append({"strategy": f"ML_LSTM2 ({latest.name})", **ml_stats})
else:
    print("No LSTM backtest stats found. Run: python run_lstm2_pipeline.py")

# Transformer
tx_jsons = sorted(REPORTS_DIR.glob("transformer_backtest_stats_*.json"))
if tx_jsons:
    latest = tx_jsons[-1]
    with latest.open("r", encoding="utf-8") as f:
        tx_stats = json.load(f)
    results.append({"strategy": f"ML_Transformer ({latest.name})", **tx_stats})
else:
    print("No Transformer backtest stats found. Run: python run_transformer_pipeline.py")

summary = pd.DataFrame(results)
cols = [
    "strategy",
    "total_return",
    "annualized_return",
    "annualized_volatility",
    "sharpe_ratio",
    "max_drawdown",
    "coverage",
    "transaction_cost_bp",
]
summary = summary[[c for c in cols if c in summary.columns]]
summary.sort_values("sharpe_ratio", ascending=False)


,strategy,total_return,annualized_return,annualized_volatility,sharpe_ratio,max_drawdown,coverage,transaction_cost_bp
1,RSI,0.132495,0.043704,0.033386,1.297697,-0.002906,0.010914,2.0
2,Bollinger(MR),0.073539,0.024696,0.052089,0.494399,-0.099792,0.046385,2.0
3,DualMA(10/50),0.107530,0.035736,0.180615,0.286411,-0.200957,0.491132,2.0
0,MACD,0.060674,0.020457,0.162780,0.205779,-0.248778,0.469304,2.0
5,"VolTargetMom(60,20)",0.038582,0.013100,0.106919,0.175383,-0.107425,0.915416,2.0
8,ML_Transformer (transformer_backtest_stats_202...,0.009428,0.009428,0.217252,0.148240,-0.173237,0.623016,2.0
4,Donchian(20/10),-0.041689,-0.014533,0.143467,-0.030190,-0.227822,0.345157,2.0
6,"BollSqueeze(20,q0.2)",-0.026519,-0.009197,0.096998,-0.046318,-0.197867,0.095498,2.0
7,ML_LSTM2 (lstm2_backtest_stats_20251229_183712...,-0.062288,-0.062288,0.103274,-0.571090,-0.134375,0.253968,2.0
